In [9]:
import cv2
import numpy as np
import os
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from scipy.cluster.vq import vq
import matplotlib as plt
# Load the dataset directory path
data_dir = "M:/SIFT/small_data"
image_paths = []
labels = []

# Load images and assign labels based on folder names
for label in os.listdir(data_dir):
    class_dir = os.path.join(data_dir, label)
    if os.path.isdir(class_dir):
        for file in os.listdir(class_dir):
            if file.endswith(('.png', '.jpg', '.jpeg')):
                image_paths.append(os.path.join(class_dir, file))
                labels.append(int(label))  # Convert folder name to integer label

# Step 1: Compute SIFT descriptors for each image
sift = cv2.SIFT_create()
descriptors_list = []

for image_path in image_paths:
    img = cv2.imread(image_path)
    if img is None:
        print(f"Warning: Could not load image {image_path}. Skipping.")
        continue  # Skip images that can't be loaded
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
    keypoints, descriptors = sift.detectAndCompute(gray, None)
    # plt.imshow(gray)
    if descriptors is not None and len(descriptors) > 0:
        descriptors_list.append(descriptors)
    else:
        print(f"Warning: No descriptors found for image {image_path}. Skipping.")

# Check if there are valid descriptors before proceeding
if len(descriptors_list) == 0:
    raise ValueError("No valid SIFT descriptors found in the dataset. Ensure images have enough texture.")

# Step 2: Flatten all descriptors for clustering
all_descriptors = np.vstack(descriptors_list)
num_clusters = 50  # Define the number of visual words
kmeans = KMeans(n_clusters=num_clusters)
kmeans.fit(all_descriptors)
visual_words = kmeans.cluster_centers_


# Step 3: Compute histogram for each image based on the visual words
def compute_histogram(descriptors, visual_words):
    words, _ = vq(descriptors, visual_words)
    hist, _ = np.histogram(words, bins=np.arange(len(visual_words) + 1))
    return hist

features = [compute_histogram(descriptors, visual_words) for descriptors in descriptors_list]
features = np.array(features)
labels = np.array(labels)

# Step 4: Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.4, random_state=42)

# Step 5: Standardize features and train a classifier
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

clf = SVC(kernel='linear')
clf.fit(X_train, y_train)

# Step 6: Predict and compute accuracy
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")


Accuracy: 93.48%
